In [ ]:
!pip install easyocr opencv-python

In [ ]:
import easyocr
import cv2
import re
reader = easyocr.Reader(['en'])

In [ ]:
def extract_details(image_path, card_type):
    # Perform OCR on the image
    recognition_results = reader.readtext(image_path, detail=0)
    
    # Combine all recognized text into a single string for processing
    extracted_text = " ".join(recognition_results)
    
    details = {}

    if card_type.lower() == "aadhaar":
        # Aadhaar extraction logic
        name = None
        dob = None
        adh = None
        sex = None
        
        # Determine gender
        if 'female' in extracted_text.lower():
            sex = "FEMALE"
        else:
            sex = "MALE"

        # Extract DOB
        try:
            dob = re.search(r'\d{2}/\d{2}/\d{4}', extracted_text).group()
        except:
            dob = "Not Found"

        # Extract Aadhaar Number
        try:
            adh = re.search(r'\d{4}\s\d{4}\s\d{4}', extracted_text).group()
        except:
            adh = "Not Found"

        # Extract Name (scan all lines for potential names)
        lines = extracted_text.split(',')
        for line in lines:
            line_cleaned = line.strip().replace('\n', '').rstrip().lstrip()
            if re.search(r'[A-Za-z]+\s[A-Za-z]+', line_cleaned) and "DOB" not in line_cleaned and "Male" not in line_cleaned:
                name = line_cleaned
                break

        # Clean Name
        if name:
            name = name.rstrip()
            name = name.lstrip()
            name = name.replace("8", "B")
            name = name.replace("0", "D")
            name = name.replace("6", "G")
            name = name.replace("1", "I")
            name = re.sub('[^a-zA-Z ]+', '', name)
        else:
            name = "Not Found"

        details['Name'] = name if name else "Not Found"
        details['Date of Birth'] = dob if dob else "Not Found"
        details['Aadhaar Number'] = adh if adh else "Not Found"
        details['Sex'] = sex if sex else "Not Found"
    
    elif card_type.lower() == "pan":
        # PAN extraction logic (unchanged)
        name_match = re.search(r'(?:Name|नाम)\s*[:\-]?\s*([A-Z\s]+)', extracted_text)
        father_name_match = re.search(r"(?:Father's Name|पिता का नाम)\s*[:\-]?\s*([A-Z\s]+)", extracted_text)
        dob_match = re.search(r'(?:Date of Birth|जन्म की तारीख)\s*[:\-]?\s*(\d{2}/\d{2}/\d{4})', extracted_text)
        pan_number_match = re.search(r'[A-Z]{5}\d{4}[A-Z]', extracted_text)
        
        details['Name'] = name_match.group(1).strip() if name_match else "Not Found"
        details['Father\'s Name'] = father_name_match.group(1).strip() if father_name_match else "Not Found"
        details['DOB'] = dob_match.group(1).strip() if dob_match else "Not Found"
        details['PAN Number'] = pan_number_match.group(0).strip() if pan_number_match else "Not Found"

    return details

In [ ]:
# Paths to Aadhaar and PAN card images
aadhaar_image_path = "/content/aadhar.jpg"
pan_image_path = "/content/pan.jpg"

In [ ]:
# Extract Aadhaar card details
aadhaar_details = extract_details(aadhaar_image_path, "aadhaar")
print("Aadhaar Card Details:", aadhaar_details)

In [ ]:
# Extract PAN card details
pan_details = extract_details(pan_image_path, "pan")
print("PAN Card Details:", pan_details)